In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from config import api_key

import numpy as np
import requests
import json

In [3]:
# Import cities file as DataFrame
twelve_thirteen_pd = pd.read_csv("draft_data/2012-13.csv")
twelve_thirteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Important Notes
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",NaN
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",NaN
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",NaN
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,NaN


In [31]:
twelve_thirteen_pd = twelve_thirteen_pd.drop(columns=["Important Notes"])


KeyError: "['Important Notes'] not found in axis"

In [10]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twelve_thirteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twelve_thirteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twelve_thirteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twelve_thirteen_pd.loc[index, "lat"] = "NaN"
        twelve_thirteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
twelve_thirteen_pd.head()



,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Lat,Lng,Place_ID,Address,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,,,,,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",,,,,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",,,,,NaN,NaN
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",,,,,-38.2537,146.576
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,,,,,-36.5343,147.459


In [34]:
twelve_thirteen_pd = twelve_thirteen_pd[twelve_thirteen_pd['Facility Name'] != 'Corporate Total']
twelve_thirteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Lat,Lng,Place_ID,Address,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,,,,,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",,,,,-37.3946,143.639
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",,,,,-38.2537,146.576
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,,,,,-36.5343,147.459
5,AGL ENERGY LIMITED,Bogong Power Station,F,"288,687",0,911,911,,,,,-36.8047,147.224


In [20]:
# Import cities file as DataFrame
thirteen_fourteen_pd = pd.read_csv("draft_data/2013-14.csv")
fourteen_fiftenn_pd = pd.read_csv("draft_data/2014-15.csv")
fifteen_sixteen_pd = pd.read_csv("draft_data/2015-16.csv")
sixteen_seventeen_pd = pd.read_csv("draft_data/2016-17.csv", encoding = "ISO-8859-1")
seventeen_eighteen_pd = pd.read_csv("draft_data/2017-18.csv", encoding = "ISO-8859-1")
eighteen_nineteen_pd = pd.read_csv("draft_data/2018-19.csv", encoding = "ISO-8859-1")
nineteen_twenty_pd = pd.read_csv("draft_data/2019-20.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("draft_data/2020-21.csv", encoding = "ISO-8859-1")

In [21]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in thirteen_fourteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        thirteen_fourteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        thirteen_fourteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        thirteen_fourteen_pd.loc[index, "lat"] = "NaN"
        thirteen_fourteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
thirteen_fourteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t-CO2-e),Greenhouse Gas Emissions Scope 2 (t-CO2-e),Greenhouse Gas Emissions Total Emissions (t-CO2-e),Important notes,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"622,579",31,375,406,NaN,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,506,981",73,"1,173","1,246",NaN,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,129,560",104,"1,548","1,652",NaN,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,"94,469",16,20,36,NaN,-36.5343,147.459
4,AGL ENERGY LIMITED,Bogong Power Station,F,"356,830",83,881,964,NaN,-36.8047,147.224


In [22]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Scope 1 (t CO2-e),Scope 2 (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,Primary Fuel,Important Notes,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,567719.0,157700.0,19,293,312,0.0,On,NEM,Wind,NaN,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,F,ACT,213115.0,59199.0,0,15,15,0.0,On,NEM,Solar,NaN,-35.4898,149.143
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,2461803.0,683834.0,77,1144,1221,0.0,On,NEM,Wind,NaN,-37.3946,143.639
3,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,NaN,3242637.0,900733.0,96,1452,1548,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,137094.0,38082.0,5,17,22,0.0,On,NEM,Hydro,NaN,-36.5343,147.459


In [23]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fifteen_sixteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fifteen_sixteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fifteen_sixteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fifteen_sixteen_pd.loc[index, "lat"] = "NaN"
        fifteen_sixteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
fifteen_sixteen_pd.head()


,Controlling corporation,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Total Scope 1 Emissions (t CO2-e),Total Scope 2 Emissions (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"595,234","165,343",22,283,305,0.00,On,NEM,Wind,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,352,139","653,372",93,950,"1,043",0.00,On,NEM,Wind,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,NaN,"2,947,373","818,715",115,"1,233","1,348",NaN,NaN,NaN,NaN,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,"170,057","47,238",0,5,5,0.00,On,NEM,Hydro,-36.5343,147.459
4,AGL ENERGY LIMITED,Bayswater Power Station,F,NSW,"64,493,748","17,914,930","15,912,788","6,252","15,919,040",0.89,On,NEM,Black Coal,-32.3926,150.949


In [24]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in sixteen_seventeen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        sixteen_seventeen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        sixteen_seventeen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        sixteen_seventeen_pd.loc[index, "lat"] = "NaN"
        sixteen_seventeen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
sixteen_seventeen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Total Scope 1 Emissions (t CO2-e),Total Scope 2 Emissions (t CO2-e)2,Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected2,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"562,485","156,246",46,283,329,0,On,NEM,Wind,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,204,701","612,417",129,"1,080","1,209",0,On,NEM,Wind,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,-,"2,767,186","768,663",175,"1,363","1,538",-,-,-,-,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,"10,938","3,038",3,10,13,0,On,NEM,Hydro,-36.5343,147.459
4,AGL ENERGY LIMITED,Bayswater Power Station,F,NSW,"57,400,488","15,944,580","14,148,670","12,723","14,161,393",0.89,On,NEM,Black Coal,-32.3926,150.949


In [25]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "lng"] = "NaN"

seventeen_eighteen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (MWh),Total Scope 1 Emissions \n(t CO2-e),Total Scope 2 Emissions \n(t CO2-e),Total Emissions \n(t CO2-e),Emission Intensity \n(t CO2-e/ MWh),Grid Connected,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Cathedral Rocks Wind Farm,F,SA,"550,950","153,042",48,136,184,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"601,740","167,150",44,205,249,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Mt Gellibrand Wind Farm,F,VIC,"4,320","1,200",0,22,22,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,317,673","643,798",132,966,"1,098",0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,-,"3,474,683","965,190",224,"1,329","1,553",-,-,-,-,NaN,NaN


In [26]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in eighteen_nineteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        eighteen_nineteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        eighteen_nineteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        eighteen_nineteen_pd.loc[index, "lat"] = "NaN"
        eighteen_nineteen_pd.loc[index, "lng"] = "NaN"

eighteen_nineteen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (MWh),Total Scope 1 Emissions \n(t CO2-e),Total Scope 2 Emissions \n(t CO2-e),Total Emissions \n(t CO2-e),Emission Intensity \n(t CO2-e/ MWh),Grid Connected,Grid,Primary Fuel,lat,lng
0,Acciona Energy Oceania Pty. Ltd.,Cathedral Rocks Wind Farm,F,SA,"632,284","175,634",39,213,252,0,On,NEM,Wind,NaN,NaN
1,Acciona Energy Oceania Pty. Ltd.,Gunning Wind Farm,F,NSW,"592,807","164,669",44,195,239,0,On,NEM,Wind,-34.6906,149.427
2,Acciona Energy Oceania Pty. Ltd.,Mt Gellibrand Wind Farm,F,VIC,"844,669","234,630",36,864,900,0,On,NEM,Wind,NaN,NaN
3,Acciona Energy Oceania Pty. Ltd.,Waubra Wind Farm,F,VIC,"2,261,159","628,100",92,901,993,0,On,NEM,Wind,-37.3946,143.639
4,Acciona Energy Oceania Pty. Ltd.,Corporate Total,C,-,"4,330,919","1,203,033",211,"2,173","2,384",-,-,-,-,NaN,NaN


In [29]:
nineteen_twenty_pd.rename(columns = {'Facility name':'Facility Name'}, inplace = True)
nineteen_twenty_pd.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions (t CO2-e),Total emissions (t CO2-e),Emission intensity (t CO2-e/ MWh),Grid connected,Grid,Primary fuel
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"555,151","154,209",50,95,145,0,On,NEM,Wind
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"636,185","176,718",45,143,188,0,On,NEM,Wind
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,086,174","301,715",49,308,357,0,On,NEM,Wind
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,414,107","670,585",139,452,591,0,On,NEM,Wind
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,691,617","1,303,227",283,998,"1,281",-,-,-,-


In [39]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in nineteen_twenty_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        nineteen_twenty_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        nineteen_twenty_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        nineteen_twenty_pd.loc[index, "lat"] = "NaN"
        nineteen_twenty_pd.loc[index, "lng"] = "NaN"

nineteen_twenty_pd.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions (t CO2-e),Total emissions (t CO2-e),Emission intensity (t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"555,151","154,209",50,95,145,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"636,185","176,718",45,143,188,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,086,174","301,715",49,308,357,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,414,107","670,585",139,452,591,0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,691,617","1,303,227",283,998,"1,281",-,-,-,-,NaN,NaN


In [ ]:
twenty_twentyone.rename(columns = {'Facility name':'Facility Name'}, inplace = True)

In [42]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "lat"] = "NaN"
        twenty_twentyone.loc[index, "lng"] = "NaN"

twenty_twentyone.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions \n(t CO2-e),Total emissions \n(t CO2-e),Emission intensity \n(t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"512,455","142,348",47,189,236,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"575,522","159,867",48,246,294,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,018,498","282,916",60,907,967,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,339,624","649,896",158,869,"1,027",0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,446,099","1,235,027",313,"2,211","2,524",-,-,-,-,NaN,NaN


In [43]:
thirteen_fourteen_pd = thirteen_fourteen_pd[thirteen_fourteen_pd['Facility Name'] != 'Corporate Total']
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
fifteen_sixteen_pd = fifteen_sixteen_pd[fifteen_sixteen_pd['Facility Name'] != 'Corporate Total']
sixteen_seventeen_pd = sixteen_seventeen_pd[sixteen_seventeen_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
nineteen_twenty_pd = nineteen_twenty_pd[nineteen_twenty_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [45]:
twenty_twentyone.head(10)

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions \n(t CO2-e),Total emissions \n(t CO2-e),Emission intensity \n(t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"512,455","142,348",47,189,236,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"575,522","159,867",48,246,294,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,018,498","282,916",60,907,967,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,339,624","649,896",158,869,"1,027",0,On,NEM,Wind,-37.3946,143.639
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,F,QLD,"587,856","163,293",0,533,533,0,On,NEM,Solar,NaN,NaN
7,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,"7,240","2,011",15,33,48,0.01,On,NEM,Hydro,-36.5343,147.459
8,AGL ENERGY LIMITED,Barker Inlet Power Station,F,SA,"1,391,597","386,555","174,233","2,829","177,062",0.45,On,NEM,Gas,-34.8043,138.525
9,AGL ENERGY LIMITED,Bayswater Power Station,F,NSW,"51,517,714","14,310,476","12,763,444","19,122","12,782,566",0.89,On,NEM,Black Coal,-32.3926,150.949
10,AGL ENERGY LIMITED,Bogong Power Station,F,VIC,"314,481","87,356",83,656,739,0,On,NEM,Hydro,-36.8047,147.224
11,AGL ENERGY LIMITED,Broken Hill Solar Plant,F,NSW,"408,929","113,591",81,343,424,0,On,NEM,Solar,-31.986,141.393
